In [1]:
!pip install sentencepiece
!pip install datasets evaluate transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.4 MB/s eta 0:00:00
 

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/fixjs/transfer

Mounted at /content/drive
/content/drive/MyDrive/fixjs/transfer


In [3]:
import pandas as pd


data = pd.read_csv("https://drive.google.com/uc?export=download&id=19nDp1WqHLUshPP3Gl4VXf_VNHUhnziHJ")

In [4]:
from sklearn.model_selection import train_test_split


RANDOM_STATE = 420
TEST_SIZE = 0.2

train_df, test_df = train_test_split(data, test_size=TEST_SIZE, random_state=RANDOM_STATE)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=RANDOM_STATE)

In [5]:
print(f'Data: {len(data)}, Train: {len(train_df)}, Test: {len(test_df)}, Val: {len(val_df)}')

Data: 41203, Train: 29665, Test: 8241, Val: 3297


In [14]:
for col in ["hate", "privacy", "sexual", "impersonation", "illegal", "advertisement", "ai"]:
  print(f'{col}: {data[col].sum()}')

hate: 19298
privacy: 0
sexual: 11
impersonation: 0
illegal: 10
advertisement: 28
ai: 1200


In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, AdamW, get_linear_schedule_with_warmup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MultilabelDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["body"]
        labels = self.data.iloc[idx][["hate", "privacy", "sexual", "impersonation", "illegal", "advertisement", "ai"]].values.astype(float)

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )

        return {
            "input_ids": torch.tensor(inputs["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(inputs["attention_mask"], dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.float)
        }

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

train_dataset = MultilabelDataset(train_df, tokenizer)
val_dataset = MultilabelDataset(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

config = RobertaConfig.from_pretrained("roberta-base", num_labels=7)
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config=config).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 50)
criterion = nn.BCEWithLogitsLoss()

patience, early_stop_counter = 3, 0
min_val_loss = float("inf")

for epoch in range(50):
    model.train()
    train_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            preds = torch.sigmoid(logits).round()
            val_loss += loss.item()
            val_correct += (preds == labels).sum().item()
            val_total += labels.numel()

    val_loss /= len(val_loader)
    val_acc = val_correct / val_total

    print(f"Epoch {epoch + 1}/{50}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), "roberta.pt")
    else:
        early_stop_counter += 1

    if early_stop_counter >= patience:
        print("Early stopping")
        break


model.load_state_dict(torch.load("roberta.pt"))

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

Epoch 1/50, Train Loss: 0.0486, Val Loss: 0.0254, Val Acc: 0.9915
Epoch 2/50, Train Loss: 0.0254, Val Loss: 0.0237, Val Acc: 0.9912
Epoch 3/50, Train Loss: 0.0223, Val Loss: 0.0234, Val Acc: 0.9918
Epoch 4/50, Train Loss: 0.0196, Val Loss: 0.0248, Val Acc: 0.9912
Epoch 5/50, Train Loss: 0.0174, Val Loss: 0.0243, Val Acc: 0.9923


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Epoch 6/50, Train Loss: 0.0154, Val Loss: 0.0273, Val Acc: 0.9909
Early stopping
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-a9ef22414cfb>", line 109, in <cell line: 109>
    model.load_state_dict(torch.load("roberta.pt"))
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 771, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 270, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 251, in __init__
    super(_open_file, self).__init__(open(name, mode))
OSError: [Errno 107] Transport endpoint is not connected: 'roberta.pt'

During handling of the above exception, another exception occurred:

Traceback (most recent call

In [13]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, AdamW, get_linear_schedule_with_warmup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MultilabelDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["body"]
        labels = self.data.iloc[idx][["hate", "privacy", "sexual", "impersonation", "illegal", "advertisement", "ai"]].values.astype(float)

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )

        return {
            "input_ids": torch.tensor(inputs["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(inputs["attention_mask"], dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.float)
        }

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

config = RobertaConfig.from_pretrained("roberta-base", num_labels=7)
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config=config).to(device)
model.load_state_dict(torch.load("roberta.pt"))

test_dataset = MultilabelDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score
import numpy as np

model.eval()
test_loss, test_correct, test_total = 0, 0, 0
all_preds, all_labels = [], []

criterion = nn.BCEWithLogitsLoss()

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels)
        preds = torch.sigmoid(logits).round()

        all_preds.extend(preds.cpu().numpy().tolist())
        all_labels.extend(labels.cpu().numpy().tolist())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

print("Classification Report for RoBERTa:\n", classification_report(all_labels, all_preds, target_names=["hate", "privacy", "sexual", "impersonation", "illegal", "advertisement", "ai"], zero_division=0))

acc = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {acc:.2f}')

f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)
print(f'F1 score: {f1:.2f}')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Classification Report for RoBERTa:
                precision    recall  f1-score   support

         hate       0.91      0.96      0.94      3789
      privacy       0.00      0.00      0.00         0
       sexual       0.00      0.00      0.00         3
impersonation       0.00      0.00      0.00         0
      illegal       0.00      0.00      0.00         3
advertisement       0.00      0.00      0.00         6
           ai       0.96      0.92      0.94       234

    micro avg       0.92      0.96      0.94      4035
    macro avg       0.27      0.27      0.27      4035
 weighted avg       0.91      0.96      0.93      4035
  samples avg       0.47      0.47      0.47      4035

Accuracy: 0.94
F1 score: 0.93
